In [3]:
%load_ext autoreload
%autoreload 2

from nirs4all.pipeline.runner import PipelineRunner
from nirs4all.pipeline.config import PipelineConfig
from nirs4all.dataset import SpectroDataset
from sample import config as python_config
from nirs4all.dataset.loader import get_dataset
from nirs4all.controllers.registry import reset_registry
from nirs4all.controllers import *

dataset_py = get_dataset(python_config["dataset"])
print("\n", "="*200, "\nPython Dataset:\n", dataset_py)

print("\n", "="*200, "\nRunning Python Config:\n")
py_config = PipelineConfig(python_config)
# json_config = PipelineConfig("sample.json")
# yaml_config = PipelineConfig("sample.yaml")

runner = PipelineRunner(max_workers=4, continue_on_error=False)

res_dataset, history, pipeline = runner.run(py_config, dataset_py)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
>> Browsing ./sample_data
No train_group file found for ./sample_data.
No test_group file found for ./sample_data.

Python Dataset:
 FeatureBlock with 1 sources and 189 samples
Source 0: FeatureSource(shape=(189, 1, 2151), dtype=float32, processing_ids=['raw'], mean=0.46620577573776245, variance=0.1492786705493927)
Unique augmentations: [None]
Index:
shape: (189, 8)
┌─────┬────────┬────────┬───────────┬───────┬────────┬────────────┬──────────────┐
│ row ┆ sample ┆ origin ┆ partition ┆ group ┆ branch ┆ processing ┆ augmentation │
│ --- ┆ ---    ┆ ---    ┆ ---       ┆ ---   ┆ ---    ┆ ---        ┆ ---          │
│ i32 ┆ i32    ┆ i32    ┆ cat       ┆ i8    ┆ i8     ┆ cat        ┆ cat          │
╞═════╪════════╪════════╪═══════════╪═══════╪════════╪════════════╪══════════════╡
│ 0   ┆ 0      ┆ null   ┆ train     ┆ 0     ┆ 0      ┆ raw        ┆ null         │
│ 1   ┆ 1      ┆ null   ┆ train     ┆ 0     ┆

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [3]:
# import kfold
from sklearn.model_selection import KFold

# random data for kfold
X = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]
y = [1, 2, 3, 4, 5]
kf = KFold(n_splits=2, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

t = kf.split(X)
print(t)
l = list(t)
for train_index, test_index in l:
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [0 3] TEST: [1 2 4]
TRAIN: [1 2 4] TEST: [0 3]
<generator object _BaseKFold.split at 0x000002006DE75070>
TRAIN: [0 3] TEST: [1 2 4]
TRAIN: [1 2 4] TEST: [0 3]
